In [1]:
ls

gtdb-rs202.minitax.csv
gtdb-rs202.nucleotide-k31-scaled1000.3fam.csv
sqlite3-nb.ipynb


In [3]:
import pandas


In [5]:
minitax_df = pandas.read_csv('gtdb-rs202.minitax.csv')
fam_df = pandas.read_csv('gtdb-rs202.nucleotide-k31-scaled1000.3fam.csv')

In [6]:
minitax_df.head()


,ident,superkingdom,phylum,class,order,family,genus,species
0,GCF_900061175.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__Streptococcus pneumoniae
1,GCF_001329375.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__Streptococcus pneumoniae
2,GCF_001151245.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__Streptococcus pneumoniae
3,GCF_001104245.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__Streptococcus pneumoniae
4,GCF_900194725.1,d__Bacteria,p__Firmicutes,c__Bacilli,o__Lactobacillales,f__Streptococcaceae,g__Streptococcus,s__Streptococcus pneumoniae


In [8]:
fam_df[['match_name', 'query_name', 'max_containment_ani']].head()

,match_name,query_name,max_containment_ani
0,GCA_901482365.1 Streptococcus pneumoniae strai...,GCF_002950215.1 Shigella flexneri 2a strain=AT...,0.980574
1,GCF_013276505.1 Streptococcus sp. 5905 strain=...,GCF_001457635.1 Streptococcus pneumoniae strai...,0.887623
2,GCF_900104285.1 Streptococcus sp. NLAE-zl-C503...,GCF_001457635.1 Streptococcus pneumoniae strai...,0.889611
3,GCF_009496155.1 Streptococcus mitis strain=SM1...,GCF_001457635.1 Streptococcus pneumoniae strai...,0.928279
4,GCF_003944215.1 Streptococcus oralis strain=BC...,GCF_001457635.1 Streptococcus pneumoniae strai...,0.884512


## A small sqlite3 primer

In [9]:
import sqlite3

db = sqlite3.connect('example.sqlite3')
cursor = db.cursor()

In [11]:
cursor.execute('CREATE TABLE comparisons (ident1 TEXT NOT NULL, ident2 TEXT NOT NULL, ani FLOAT NOT NULL)')

In [12]:
cursor.execute('INSERT INTO comparisons (ident1, ident2, ani) VALUES (?, ?, ?)', 
               ('a', 'b', 5))

In [14]:
cursor.execute('SELECT ident1, ident2, ani FROM comparisons')

In [15]:
cursor.fetchall()

[('a', 'b', 5.0)]

In [16]:
cursor.execute('INSERT INTO comparisons (ident1, ident2, ani) VALUES (?, ?, ?)', 
               ('c', 'd', 10))

In [17]:
cursor.execute('SELECT ident1, ident2, ani FROM comparisons')

In [18]:
cursor.fetchall()

[('a', 'b', 5.0), ('c', 'd', 10.0)]

In [20]:
db.commit()

In [21]:
cursor.execute('SELECT ident1, ident2, ani FROM comparisons WHERE ani > 5')
cursor.fetchall()

[('c', 'd', 10.0)]

In [22]:
cursor.execute('SELECT ident1, ident2, ani FROM comparisons WHERE ani > 5')
for tup in cursor:
    print(tup)

('c', 'd', 10.0)


In [29]:
cursor.execute('SELECT MAX(ani) from comparisons')
cursor.fetchall()

[(10.0,)]

In [30]:
cursor.execute('SELECT MIN(ani) from comparisons')
cursor.fetchall()

[(5.0,)]

In [33]:
cursor.execute('SELECT AVG(ani) from comparisons WHERE ani > 5')
cursor.fetchall()

[(10.0,)]

In [37]:
cursor.execute('SELECT ident1, ident2, ani FROM comparisons ORDER BY ani DESC LIMIT 1')
for tup in cursor:
    print(tup)

('c', 'd', 10.0)


In [38]:
cursor.execute('CREATE TABLE lineages (ident TEXT NOT NULL, lineage TEXT NOT NULL)')

In [39]:
cursor.execute('SELECT DISTINCT ident1 FROM comparisons')
cursor.fetchall()

[('a',), ('c',)]

In [40]:
cursor.execute('SELECT DISTINCT ident2 FROM comparisons')
cursor.fetchall()

[('b',), ('d',)]

In [42]:
cursor.executemany('INSERT INTO lineages (ident, lineage) VALUES (?, ?)',
                   (('a', 'lin1'), ('b', 'lin2'), ('c', 'lin1'), ('d', 'lin3')))

In [48]:
cursor.execute('''SELECT ani FROM comparisons, lineages
WHERE comparisons.ident1=lineages.ident
AND comparisons.ident2=lineages.ident''')
cursor.fetchall()

[]

## trying again

In [55]:
db = sqlite3.connect(':memory:')
c = db.cursor()

c.execute('''
CREATE TABLE comparisons (
ident1 TEXT NOT NULL,
ident2 TEXT NOT NULL,
lca_rank TEXT NOT NULL,
lca_name TEXT NOT NULL,
ani FLOAT NOT NULL
)
''')

c.executemany('''
INSERT INTO comparisons (ident1, ident2, lca_rank, lca_name, ani) VALUES (?, ?, ?, ?, ?)
''',
            (('a', 'b', 'phylum', 'foo', 0.1),
             ('a', 'd', 'phylum', 'foofiz', 0.2),
             ('a', 'c', 'family', 'fiz', 0.4)))

In [56]:
c.execute('''
SELECT MIN(ani), AVG(ani), MAX(ani) FROM comparisons WHERE lca_rank=?
''', ('phylum',))
c.fetchall()

[(0.1, 0.15000000000000002, 0.2)]